In [ ]:
import numpy as np
from scipy.optimize import least_squares
import itertools as it

In [ ]:
def resolver_matriz_A_robusto(X_target, n=6, pares=None, tentativas=200, chute_scale=1.0, method='trf', nonneg=True):
    """
    Recupera uma matriz A (n x n) tal que, para os TRIPLOS em `pares`,
    a matriz X calculada via PERMANENTE de submatriz 3x3 se aproxime de X_target.

    Observação: agora cada equação usa uma submatriz 3x3 extraída de A,
    com linhas (r1,r2,r3) e colunas (c1,c2,c3). A permanente 3x3 soma
    todos os produtos de A[r_k, c_{perm(k)}] para permutações de 3 elementos,
    sem sinais alternados.
    """
    import numpy as np
    from scipy.optimize import least_squares
    import itertools as it

    def permanent_3x3(M):
        # M é array 3x3; permanente = soma sobre todas as 3! permutações
        perms = it.permutations([0,1,2])
        s = 0.0
        for p in perms:
            s += M[0, p[0]] * M[1, p[1]] * M[2, p[2]]
        return s

    if pares is None:

        pares = [
            (0, 2, 4),
            (0, 2, 5),
            (0, 3, 4),
            (0, 3, 5),
            (1, 2, 4),
            (1, 2, 5),
            (1, 3, 4),
            (1, 3, 5),
        ]

    m = len(pares)
    assert X_target.shape == (m, m), f"X_target deve ser {m}x{m}, recebido {X_target.shape}"

    def residuos(vars_flat):
        A = vars_flat.reshape(n, n)
        diffs = []
        for i in range(m):
            r1, r2, r3 = pares[i]
            for j in range(m):
                c1, c2, c3 = pares[j]
                subM = np.array([
                    [A[r1, c1], A[r1, c2], A[r1, c3]],
                    [A[r2, c1], A[r2, c2], A[r2, c3]],
                    [A[r3, c1], A[r3, c2], A[r3, c3]],
                ])
                val_calc = permanent_3x3(subM)
                diffs.append(val_calc - X_target[i, j])
        return np.array(diffs)

    print(f"Iniciando busca n={n}, {m} triplos (X {m}x{m}), até {tentativas} tentativas... (nonneg={nonneg})")

    melhor_erro = float('inf')
    melhor_solucao = None

    lb = 0.0 if nonneg else -np.inf
    ub = np.inf

    for t in range(tentativas):
        chute = np.random.uniform(0 if nonneg else -chute_scale, chute_scale, n*n)
        if nonneg:
            chute = np.clip(chute, 0, None)
        resultado = least_squares(
            residuos,
            chute,
            method=method,
            bounds=(lb*np.ones(n*n), ub*np.ones(n*n)) if nonneg else (-np.inf*np.ones(n*n), np.inf*np.ones(n*n))
        )
        erro_atual = resultado.cost
        if erro_atual < 1e-12:
            print(f"-> Solução encontrada na tentativa {t+1}!")
            return resultado.x.reshape(n, n)
        if erro_atual < melhor_erro:
            melhor_erro = erro_atual
            melhor_solucao = resultado.x.reshape(n, n)

    print(f"Aviso: Solução exata não encontrada. Melhor erro residual: {melhor_erro}")
    return melhor_solucao


In [54]:
def result_to_matrix(flat_arr, n):
    return flat_arr.reshape(n, n)

In [55]:
# --- TESTE (n=6, X 8x8, usando TRIPLOS e permanente 3x3) ---

# 1. Matriz Real (A original) 6x6 não-negativa
A_real = np.array([
    [0.8, 0.1, 0.7, 0.3, 0.5, 0.2],
    [0.6, 0.9, 0.2, 0.4, 0.3, 0.7],
    [0.2, 0.5, 0.6, 0.1, 0.4, 0.8],
    [0.7, 0.3, 0.5, 0.6, 0.2, 0.1],
    [0.3, 0.4, 0.1, 0.2, 0.9, 0.5],
    [0.5, 0.2, 0.8, 0.3, 0.6, 0.4]
], dtype=float)

n = 6
# TRIPLOS conforme especificado
pares = [
    (0, 2, 4),
    (0, 2, 5),
    (0, 3, 4),
    (0, 3, 5),
    (1, 2, 4),
    (1, 2, 5),
    (1, 3, 4),
    (1, 3, 5),
]
print("Triplos usados (8):", pares)

Triplos usados (8): [(0, 2, 4), (0, 2, 5), (0, 3, 4), (0, 3, 5), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5)]


In [ ]:
# 2. Gerar X (Target) 8x8 a partir de A_real e dos triplos (permanente 3x3)
def permanent_3x3(M):
    s = 0.0
    for p in it.permutations([0,1,2]):
        s += M[0, p[0]] * M[1, p[1]] * M[2, p[2]]
    return s

m = len(pares)
X_input = np.zeros((m, m))
for i in range(m):
    r1, r2, r3 = pares[i]
    for j in range(m):
        c1, c2, c3 = pares[j]
        subM = np.array([
            [A_real[r1, c1], A_real[r1, c2], A_real[r1, c3]],
            [A_real[r2, c1], A_real[r2, c2], A_real[r2, c3]],
            [A_real[r3, c1], A_real[r3, c2], A_real[r3, c3]],
        ])
        X_input[i, j] = permanent_3x3(subM)

print("Matriz X (Alvo) 8x8 (permanente 3x3):\n", np.round(X_input, decimals=3))
print("Min/Max de X_input:", np.min(X_input), np.max(X_input))
print("-" * 30)

Matriz X (Alvo) 8x8 (permanente 3x3):
 [[0.774 0.582 0.261 0.284 0.63  0.495 0.27  0.22 ]
 [0.998 1.132 0.295 0.39  0.594 0.444 0.217 0.17 ]
 [0.969 0.518 0.831 0.434 0.407 0.205 0.313 0.149]
 [1.137 0.617 0.747 0.417 0.37  0.194 0.213 0.111]
 [0.468 0.436 0.243 0.311 0.731 0.659 0.439 0.515]
 [0.61  0.946 0.269 0.437 0.844 1.228 0.365 0.515]
 [0.486 0.386 0.72  0.568 0.562 0.433 0.752 0.574]
 [0.623 0.801 0.613 0.651 0.56  0.518 0.529 0.446]]
Min/Max de X_input: 0.11100000000000002 1.2280000000000002
------------------------------


In [61]:
Toffoli = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0]])

In [ ]:
# 3. Tentar recuperar A (6x6) com A>=0
A_encontrada = resolver_matriz_A_robusto(
    Toffoli, n=n, pares=pares, tentativas=300, chute_scale=1.0, method='trf', nonneg=False
)

Iniciando busca n=6, 8 triplos (X 8x8), até 300 tentativas... (nonneg=False)
-> Solução encontrada na tentativa 1!

Matriz A Encontrada (6x6):
 [[ 1.455 -0.     0.    -0.    -1.105  1.105]
 [ 0.     1.455 -0.     0.     0.    -0.   ]
 [ 2.059 -0.    -0.562 -0.     1.564 -1.564]
 [ 0.    -0.     0.    -0.562 -0.    -0.   ]
 [ 1.611 -0.     0.439 -0.    -0.    -1.224]
 [-1.611 -0.    -0.439  0.    -1.224 -0.   ]]
-> Solução encontrada na tentativa 1!

Matriz A Encontrada (6x6):
 [[ 1.455 -0.     0.    -0.    -1.105  1.105]
 [ 0.     1.455 -0.     0.     0.    -0.   ]
 [ 2.059 -0.    -0.562 -0.     1.564 -1.564]
 [ 0.    -0.     0.    -0.562 -0.    -0.   ]
 [ 1.611 -0.     0.439 -0.    -0.    -1.224]
 [-1.611 -0.    -0.439  0.    -1.224 -0.   ]]


In [58]:
# 4. Validar o resultado recalculando X a partir da A encontrada (permanente 3x3)
import itertools as it

def permanent_3x3(M):
    s = 0.0
    for p in it.permutations([0,1,2]):
        s += M[0, p[0]] * M[1, p[1]] * M[2, p[2]]
    return s

m = len(pares)
X_check = np.zeros((m, m))
for i in range(m):
    r1, r2, r3 = pares[i]
    for j in range(m):
        c1, c2, c3 = pares[j]
        subM = np.array([
            [A_encontrada[r1, c1], A_encontrada[r1, c2], A_encontrada[r1, c3]],
            [A_encontrada[r2, c1], A_encontrada[r2, c2], A_encontrada[r2, c3]],
            [A_encontrada[r3, c1], A_encontrada[r3, c2], A_encontrada[r3, c3]],
        ])
        X_check[i, j] = permanent_3x3(subM)

print("Min/Max de X_check:", np.min(X_check), np.max(X_check))
erro_maximo = np.max(np.abs(X_input - X_check))
print(f"\nErro Máximo (X_alvo - X_recalculado): {erro_maximo:.10f}")

if erro_maximo < 1e-5:
    print("SUCESSO: A matriz encontrada gera exatamente a matriz X (8x8).")
else:
    print("FALHA: A matriz encontrada não gera a matriz X.")

Min/Max de X_check: 0.11100000000000038 1.2280000000000055

Erro Máximo (X_alvo - X_recalculado): 0.0000000000
SUCESSO: A matriz encontrada gera exatamente a matriz X (8x8).


In [59]:
print(np.round(X_check, decimals=2))
print("\n", np.round(X_input, decimals=2))

[[0.77 0.58 0.26 0.28 0.63 0.49 0.27 0.22]
 [1.   1.13 0.3  0.39 0.59 0.44 0.22 0.17]
 [0.97 0.52 0.83 0.43 0.41 0.21 0.31 0.15]
 [1.14 0.62 0.75 0.42 0.37 0.19 0.21 0.11]
 [0.47 0.44 0.24 0.31 0.73 0.66 0.44 0.52]
 [0.61 0.95 0.27 0.44 0.84 1.23 0.37 0.52]
 [0.49 0.39 0.72 0.57 0.56 0.43 0.75 0.57]
 [0.62 0.8  0.61 0.65 0.56 0.52 0.53 0.45]]

 [[0.77 0.58 0.26 0.28 0.63 0.49 0.27 0.22]
 [1.   1.13 0.3  0.39 0.59 0.44 0.22 0.17]
 [0.97 0.52 0.83 0.43 0.41 0.2  0.31 0.15]
 [1.14 0.62 0.75 0.42 0.37 0.19 0.21 0.11]
 [0.47 0.44 0.24 0.31 0.73 0.66 0.44 0.52]
 [0.61 0.95 0.27 0.44 0.84 1.23 0.37 0.52]
 [0.49 0.39 0.72 0.57 0.56 0.43 0.75 0.57]
 [0.62 0.8  0.61 0.65 0.56 0.52 0.53 0.45]]
